In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from matplotlib.colors import LinearSegmentedColormap
import os

# Mandelbrot set function
def mandelbrot_set(width, height, zoom, max_iter):
    x = np.linspace(-2.0, 1.0, width) / zoom
    y = np.linspace(-1.5, 1.5, height) / zoom
    X, Y = np.meshgrid(x, y)
    C = X + 1j * Y
    Z = np.zeros_like(C)
    img = np.zeros(C.shape, dtype=int)

    for i in range(max_iter):
        mask = np.abs(Z) <= 2
        Z[mask] = Z[mask] ** 2 + C[mask]
        img[mask] += 1
    return img

# Julia set function
def julia_set(width, height, zoom, c, max_iter):
    x = np.linspace(-1.5, 1.5, width) / zoom
    y = np.linspace(-1.5, 1.5, height) / zoom
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y
    img = np.zeros(Z.shape, dtype=int)

    for i in range(max_iter):
        mask = np.abs(Z) <= 2
        Z[mask] = Z[mask] ** 2 + c
        img[mask] += 1
    return img

# Create neon colormap
def create_neon_colormap():
    colors = [
        (0.0, '#000000'),       # Black
        (0.2, '#8A2BE2'),       # BlueViolet
        (0.4, '#4B0082'),       # Indigo
        (0.6, '#00FFFF'),       # Cyan
        (0.8, '#FF00FF'),       # Magenta
        (1.0, '#FFFFFF')        # White
    ]
    cmap = LinearSegmentedColormap.from_list('neon', colors)
    return cmap

# Generate frames
def generate_frames(num_frames, width=800, height=800, zoom=1, max_iter=300):
    frames = []
    cmap = create_neon_colormap()

    # Create a temporary directory for frames
    temp_dir = './temp_frames'
    os.makedirs(temp_dir, exist_ok=True)

    # Transition parameters
    c_start = complex(-0.7, 0.27015)
    c_end = complex(-0.835, -0.2321)

    for i in range(num_frames):
        t = i / (num_frames - 1)
        
        if t < 0.5:  # First half: Julia set
            c = c_start * (1 - 2 * t) + c_end * (2 * t)
            img = julia_set(width, height, zoom, c, max_iter)
        else:  # Second half: Blending Julia set into Mandelbrot
            mandelbrot_weight = (t - 0.5) * 2
            julia_weight = 1 - mandelbrot_weight
            c = c_end  # Fixed c for Mandelbrot
            julia_img = julia_set(width, height, zoom, c, max_iter)
            mandelbrot_img = mandelbrot_set(width, height, zoom, max_iter)
            img = (julia_weight * julia_img + mandelbrot_weight * mandelbrot_img).astype(int)

        img_normalized = img / img.max()
        frame_filename = os.path.join(temp_dir, f'temp_frame_{i}.png')
        save_frame(img_normalized, cmap, frame_filename)
        frames.append(imageio.imread(frame_filename))

    # Cleanup temp directory
    for file in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, file))
    os.rmdir(temp_dir)
    
    return frames

# Helper function to save a frame
def save_frame(img, cmap, filename):
    plt.figure(figsize=(8, 8), dpi=300)
    plt.imshow(img, cmap=cmap, extent=(-2, 1, -1.5, 1.5))
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

# Create GIF from frames
def create_gif(frames, filename='julia_to_mandelbrot.gif', duration=0.05, pause_duration=1.0):
    # Add pause by duplicating the last frame
    pause_frames = int(pause_duration / duration)
    frames.extend([frames[-1]] * pause_frames)

    # Save the GIF
    imageio.mimsave(filename, frames, duration=duration)

# Main function
def main():
    num_frames = 200  # Adjust for smoother transition
    frames = generate_frames(num_frames)
    gif_path = os.path.join(os.getcwd(), 'julia_to_mandelbrot.gif')
    create_gif(frames, filename=gif_path, duration=0.05, pause_duration=1.0)
    print(f"GIF created successfully! You can find it here: {gif_path}")

# Run the script in Jupyter Notebook
main()

/var/folders/1h/25ycvzv1075f8yjvmry43bl00000gn/T/ipykernel_43890/577817488.py:79: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_filename))


GIF created successfully! You can find it here: /Users/dr_awkward/Programming/Fun/julia_to_mandelbrot.gif


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import os

# Define Quaternion Julia Set rendering
def quaternion_julia(width, height, q_start, q_end, num_frames, color_scheme, max_iter=20, zoom=1.5):
    """
    Generate Quaternion Julia Set frames for a GIF.
    
    Parameters:
        width (int): Width of the image.
        height (int): Height of the image.
        q_start (tuple): Starting quaternion (w, x, y, z).
        q_end (tuple): Ending quaternion (w, x, y, z).
        num_frames (int): Number of frames for the animation.
        color_scheme (str): Color scheme name.
        max_iter (int): Maximum iterations for escape.
        zoom (float): Zoom level.
    """
    frames = []
    cmap = select_colormap(color_scheme)
    
    for i in range(num_frames):
        t = i / (num_frames - 1)
        q = tuple(q_start[j] * (1 - t) + q_end[j] * t for j in range(4))  # Interpolate quaternion states
        
        # Generate the Julia set
        x = np.linspace(-zoom, zoom, width)
        y = np.linspace(-zoom, zoom, height)
        X, Y = np.meshgrid(x, y)
        Z = X + 1j * Y
        
        # Iterate the quaternion Julia set
        img = np.zeros(Z.shape, dtype=int)
        for iter_count in range(max_iter):
            Z = Z ** 2 + (q[0] + q[1]*1j + q[2]*1j**2 + q[3]*1j**3)
            mask = np.abs(Z) < 2
            img[mask] += 1
        
        # Normalize for coloring
        img_normalized = img / max_iter
        
        # Save frame
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(img_normalized, cmap=cmap, extent=(-zoom, zoom, -zoom, zoom))
        ax.axis('off')
        
        # Save to temporary file
        frame_path = f"frame_{i}.png"
        plt.savefig(frame_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        frames.append(imageio.imread(frame_path))
        os.remove(frame_path)
    
    return frames

# Select a colormap
def select_colormap(color_scheme):
    if color_scheme == "ultraviolet":
        return LinearSegmentedColormap.from_list("ultraviolet", ["#000000", "#8A2BE2", "#9400D3", "#FF00FF", "#FFFFFF"])
    elif color_scheme == "neon":
        return LinearSegmentedColormap.from_list("neon", ["#000000", "#00FFFF", "#00FF00", "#FFFF00", "#FFFFFF"])
    elif color_scheme == "chartreuse":
        return LinearSegmentedColormap.from_list("chartreuse", ["#000000", "#7FFF00", "#ADFF2F", "#FFFFFF"])
    elif color_scheme == "fiery_sunset":
        return LinearSegmentedColormap.from_list("fiery_sunset", ["#000000", "#FF4500", "#FF6347", "#FFD700", "#FFFFFF"])
    elif color_scheme == "deep_ocean":
        return LinearSegmentedColormap.from_list("deep_ocean", ["#000000", "#00008B", "#1E90FF", "#87CEEB", "#FFFFFF"])
    else:
        raise ValueError(f"Unknown color scheme: {color_scheme}")

# Create GIF from frames
def create_gif(frames, output_filename, frame_duration=0.05):
    """
    Create a GIF from a list of frames.
    
    Parameters:
        frames (list): List of image frames.
        output_filename (str): Output GIF file name.
        frame_duration (float): Duration of each frame in seconds.
    """
    imageio.mimsave(output_filename, frames, duration=frame_duration)

# Main function
def main():
    # Parameters
    width, height = 400, 400
    num_frames = 30  # 0.5 seconds at 60 FPS
    q_start = (0.355, 0.355, 0.355, 0.355)
    q_end = (0.4, -0.4, -0.4, 0.4)
    color_schemes = ["ultraviolet", "neon", "chartreuse", "fiery_sunset", "deep_ocean"]
    output_folder = "./output_gifs"
    os.makedirs(output_folder, exist_ok=True)
    
    # Generate GIFs for each color scheme
    for color_scheme in color_schemes:
        print(f"Generating GIF for color scheme: {color_scheme}")
        frames = quaternion_julia(width, height, q_start, q_end, num_frames, color_scheme)
        output_filename = os.path.join(output_folder, f"quaternion_julia_{color_scheme}.gif")
        create_gif(frames, output_filename, frame_duration=0.05)
        print(f"Saved GIF: {output_filename}")

# Run the program
if __name__ == "__main__":
    main()

Generating GIF for color scheme: ultraviolet


/var/folders/1h/25ycvzv1075f8yjvmry43bl00000gn/T/ipykernel_44527/1935847444.py:39: RuntimeWarning: overflow encountered in square
  Z = Z ** 2 + (q[0] + q[1]*1j + q[2]*1j**2 + q[3]*1j**3)
/var/folders/1h/25ycvzv1075f8yjvmry43bl00000gn/T/ipykernel_44527/1935847444.py:39: RuntimeWarning: invalid value encountered in square
  Z = Z ** 2 + (q[0] + q[1]*1j + q[2]*1j**2 + q[3]*1j**3)
/var/folders/1h/25ycvzv1075f8yjvmry43bl00000gn/T/ipykernel_44527/1935847444.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_path))


Saved GIF: ./output_gifs/quaternion_julia_ultraviolet.gif
Generating GIF for color scheme: neon
Saved GIF: ./output_gifs/quaternion_julia_neon.gif
Generating GIF for color scheme: chartreuse
Saved GIF: ./output_gifs/quaternion_julia_chartreuse.gif
Generating GIF for color scheme: fiery_sunset
Saved GIF: ./output_gifs/quaternion_julia_fiery_sunset.gif
Generating GIF for color scheme: deep_ocean
Saved GIF: ./output_gifs/quaternion_julia_deep_ocean.gif
